<a href="https://colab.research.google.com/github/vijaychandrab-hub/AI-Driven-Study-Schedule-Generator/blob/main/AI-Driven-Study-Schedule-Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json, csv, math
from datetime import datetime, timedelta

# -----------------------------------------------------------
# 1. Helper functions
# -----------------------------------------------------------
def daterange(start, end):
    """Yield each date between start and end (inclusive)."""
    for n in range((end - start).days + 1):
        yield start + timedelta(days=n)

# -----------------------------------------------------------
# 2. Core scheduling logic
# -----------------------------------------------------------
def compute_priorities(topics):
    """Compute normalized priority score for each topic."""
    for t in topics:
        t["priority"] = (1 - t.get("mastery", 0)) * t.get("difficulty", 1) * t.get("importance", 1)
    total = sum(t["priority"] for t in topics)
    for t in topics:
        t["priority"] = t["priority"] / total if total else 1 / len(topics)
    return topics

def allocate_time(topics, total_minutes):
    """Allocate total minutes across topics proportionally."""
    for t in topics:
        t["allocated_minutes"] = max(30, round(t["priority"] * total_minutes))
    return topics

def plan_schedule(user):
    start = datetime.strptime(user["start_date"], "%Y-%m-%d").date()
    end = datetime.strptime(user["deadline"], "%Y-%m-%d").date()
    weekdays = user["daily_hours_by_weekday"]  # {0..6: hours}
    topics = compute_priorities(user["topics"])

    # Total available minutes
    total_minutes = 0
    minutes_per_day = {}
    for d in daterange(start, end):
        hrs = weekdays.get(d.weekday(), 0)
        minutes_per_day[d] = int(hrs * 60)
        total_minutes += minutes_per_day[d]
    if total_minutes <= 0:
        raise ValueError("No study hours defined for this date range.")

    topics = allocate_time(topics, total_minutes)

    # Assign sessions greedily by priority
    schedule = []
    topics.sort(key=lambda x: x["priority"], reverse=True)
    for t in topics:
        remaining = t["allocated_minutes"]
        for d in daterange(start, end):
            if remaining <= 0:
                break
            avail = minutes_per_day[d]
            if avail < 30:
                continue
            chunk = min(90, remaining, avail)  # up to 1.5h per block
            chunk = math.floor(chunk / 5) * 5
            schedule.append({
                "date": d.isoformat(),
                "subject": t["subject"],
                "topic": t["topic"],
                "minutes": chunk,
                "session_type": "learn"
            })
            minutes_per_day[d] -= chunk
            remaining -= chunk

    # Spaced repetition (1, 3, 7 days later)
    review_offsets = [1, 3, 7]
    first_dates = {}
    for s in schedule:
        key = (s["subject"], s["topic"])
        if key not in first_dates:
            first_dates[key] = datetime.strptime(s["date"], "%Y-%m-%d").date()

    for (sub, top), first in first_dates.items():
        for off in review_offsets:
            rd = first + timedelta(days=off)
            if rd > end:
                continue
            schedule.append({
                "date": rd.isoformat(),
                "subject": sub,
                "topic": top,
                "minutes": 20,
                "session_type": f"review+{off}d"
            })

    schedule.sort(key=lambda x: (x["date"], x["subject"]))
    return schedule

# -----------------------------------------------------------
# 3. Export functions
# -----------------------------------------------------------
def save_schedule(schedule):
    with open("schedule.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["date","subject","topic","minutes","session_type"])
        writer.writeheader()
        writer.writerows(schedule)
    with open("schedule.json", "w", encoding="utf-8") as jf:
        json.dump(schedule, jf, indent=2)
    print("✅ Files saved: schedule.csv, schedule.json")

# -----------------------------------------------------------
# 4. Example user input
# -----------------------------------------------------------
user_input = {
    "start_date": "2025-10-16",
    "deadline": "2025-12-10",
    "daily_hours_by_weekday": {
        0: 2.5,  # Monday
        1: 2.5,
        2: 2.5,
        3: 2.5,
        4: 2.5,
        5: 4.0,  # Saturday
        6: 0.0   # Sunday off
    },
    "topics": [
        {"subject": "Math", "topic": "Algebra", "difficulty": 4, "mastery": 0.3, "importance": 1.0},
        {"subject": "Math", "topic": "Calculus", "difficulty": 5, "mastery": 0.2, "importance": 1.0},
        {"subject": "Physics", "topic": "Mechanics", "difficulty": 4, "mastery": 0.5, "importance": 0.9},
        {"subject": "Chemistry", "topic": "Organic", "difficulty": 3, "mastery": 0.6, "importance": 0.8},
        {"subject": "Chemistry", "topic": "Inorganic", "difficulty": 2, "mastery": 0.7, "importance": 0.6},
    ]
}

# -----------------------------------------------------------
# 5. Run generator
# -----------------------------------------------------------
if __name__ == "__main__":
    schedule = plan_schedule(user_input)
    print(f"Generated {len(schedule)} study sessions.\nSample:")
    for s in schedule[:10]:
        print(f"{s['date']}: {s['subject']} - {s['topic']} ({s['minutes']} min, {s['session_type']})")
    save_schedule(schedule)

Generated 133 study sessions.
Sample:
2025-10-16: Math - Calculus (90 min, learn)
2025-10-16: Math - Algebra (60 min, learn)
2025-10-17: Math - Calculus (90 min, learn)
2025-10-17: Math - Algebra (60 min, learn)
2025-10-17: Math - Calculus (20 min, review+1d)
2025-10-17: Math - Algebra (20 min, review+1d)
2025-10-18: Math - Calculus (90 min, learn)
2025-10-18: Math - Algebra (90 min, learn)
2025-10-18: Physics - Mechanics (60 min, learn)
2025-10-19: Math - Calculus (20 min, review+3d)
✅ Files saved: schedule.csv, schedule.json
